# Imports

In [2]:
# General imports
import os
import sys
import random 
import functools

import cv2
import piq
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.auto import tqdm
import matplotlib.pyplot as plt 
import albumentations as albu
import albumentations.pytorch as albu_pt

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

# Fix to be able to import python modules inside a notebook
os.chdir('..')

# Useful extensions
# %load_ext watermark
# %watermark -v -n -m -p numpy,torch,albumentations,photosynthesis_metrics

# %load_ext autoreload
# %autoreload 2

# Nice plot formating
%matplotlib inline

In [4]:
!ls

configs  logs	   models     old_logs	 reports	   src	  train.py
data	 Makefile  notebooks  README.md  requirements.txt  tests  WORKPLAN.md


Training GAN's experiments:

GAN runner
GAN callback to support custom metrics

GAN model (from Skoltech DL course)



# Parse model results to get best checkpoints

In [15]:
from src.features.models import MODEL_FROM_NAME

def get_best_model(task: str, metric: str, mode: str = "min", root: str = "logs") -> nn.Module:
    """Returns best model for specific task and metric
    Args:
        task: One of {`sr`, `denoise`, `deblur`}
        metric: Metric name
        mode: Find minimum or maximum value
        root: Path to folder with logs
    """
    task_folders = []
    for folder_name in os.listdir(ROOT):
        if task == folder_name.split("_")[0]:
            task_folders.append(folder_name)
            
    if mode == "min":
        best = np.inf
        monitor_op = np.less
    elif mode == "max":
        best = - np.inf
        monitor_op = np.greater
    else:
        raise ValueError(f"Mode {mode} not defined!")
    
    model_name, model_weights = None, None

    for folder in task_folders:
        # Load checkpoint
        checkpoint_path = os.path.join(ROOT, folder, f"model_{METRIC}.chpn")
        checkpoint = torch.load(checkpoint_path)
        current = checkpoint["value"]
        if monitor_op(current, best):
            best = current
            model_name = folder.split("_")[2]
            model_weights = checkpoint["state_dict"]
        
    # Load model
    model = MODEL_FROM_NAME[model_name]()
    model.load_state_dict(model_weights)
    print(f"Found model with best {metric} value {best:.4f}")
    return model
       
model = get_best_model(task="denoise", metric="psnr", mode="max")

Found model with best psnr value 25.3820


In [ ]:
# Inference 

['denoise_div2k_dncnn_256_l2', 'denoise_div2k_dncnn_256_l1_ms-ssim', 'denoise_div2k_dncnn_128_l1', 'denoise_div2k_dncnn_256_l1']
logs/denoise_div2k_dncnn_256_l2/model_psnr.chpn
{'epoch': 61, 'value': 25.333821160452707}
logs/denoise_div2k_dncnn_256_l1_ms-ssim/model_psnr.chpn
{'epoch': 69, 'value': 24.624186652047293}
logs/denoise_div2k_dncnn_128_l1/model_psnr.chpn
{'epoch': 46, 'value': 25.080595896794247}
logs/denoise_div2k_dncnn_256_l1/model_psnr.chpn
{'epoch': 55, 'value': 25.382037026541575}


logs ['denoise_div2k_dncnn_256_l2', 'denoise_div2k_dncnn_256_l1_ms-ssim', 'deblur_div2k_dncnn_256_l1_1', 'deblur_div2k_dncnn_256_l1', 'deblur_div2k_dncnn_128_l1']
logs ['denoise_div2k_dncnn_256_l2', 'denoise_div2k_dncnn_256_l1_ms-ssim', 'deblur_div2k_dncnn_256_l1_1', 'deblur_div2k_dncnn_256_l1', 'deblur_div2k_dncnn_128_l1']


In [84]:
metrics = ["psnr", "ssim", "ms-ssim", "ms-gmsdc", "fsimc", "vsi", "mdsi", "vifp",
"content_vgg16_ap", "style_vgg16", "lpips", "dists", "brisque",
"is_metric", "is", "kid", "fid", "msid"]
print(len(metrics))

tasks = 3

images = 20

18


In [31]:
from src.data import crop_patches

target = torch.rand(4, 3, 128, 128)
target_patches = crop_patches(target, size=96, stride=32)
print(target_patches.shape)

dataset = torch.utils.data.TensorDataset(target_patches)
print(dataset[0][0].shape)

torch.Size([16, 3, 96, 96])
torch.Size([3, 96, 96])


In [26]:
class PSNR(torch.nn.modules.loss._Loss):
    def __init(self, data_range=1.0, reduction='mean', convert_to_greyscale: bool = False):
        super().__init__()
        
        self.metric = functools.partial(
            piq.psnr, data_range=data_range, reduction=reduction, convert_to_greyscale=convert_to_greyscale)

    def forward(self, prediction: torch.Tensor, target: torch.Tensor):
        self.metric(prediction, target)


kwargs = {'data_range': 1.0, 'convert_to_greyscale': False},
PSNR(kwargs)

PSNR()

In [24]:
issubclass(piq.GMSDLoss, torch.nn.modules.loss._Loss)
issubclass(piq.fsim, torch.nn.modules.loss._Loss)

TypeError: issubclass() arg 1 must be a class

In [45]:
a = torch.rand(2, 3, 4, 4, 2)
res = torch.split(a, split_size_or_sections=1, dim=4)
res[1].shape

torch.Size([2, 3, 4, 4, 1])

In [15]:
piq.multi_scale_ssim?

In [13]:
prediction = torch.rand(4, 3, 128, 128)
piq.MultiScaleGMSDLoss()(prediction, prediction)

tensor(0.)

In [4]:
piq.ContentLoss?

In [2]:
a = "model_{ep}_{metric:.2f}.chpn"
a.format(ep=3, metric=0.234556436)

'model_3_0.23.chpn'

In [12]:
kwargs = {
    "feature_extractor": 'vgg19',
    "layers": ['conv1_2', 'conv2_2', 'conv3_4', 'conv4_4', 'conv5_4'],
    "weights": [0.2, 0.2, 0.2, 0.2, 0.2],
    "normalize_features": True,
    "reduction": 'none',
}

kwargs.pop("feature_extractor")
kwargs

{'layers': ['conv1_2', 'conv2_2', 'conv3_4', 'conv4_4', 'conv5_4'],
 'weights': [0.2, 0.2, 0.2, 0.2, 0.2],
 'normalize_features': True,
 'reduction': 'none'}

In [ ]:
        # DummyAverageMeter is added to the end, so just delete last part
        counter = 0
        for i, metric in enumerate(self.state.metric_meters):
            if isinstance(metric, AverageMeter):
                metric.reset()
            else:
                counter += 1
        self.state.metric_meters = self.state.metric_meters[:-counter]

In [67]:
reduction = 'none'
f = functools.partial(piq.vif_p, reduction=reduction)
f.name = "test"
print(f, f.name)

functools.partial(<function vif_p at 0x7f59f1780d08>, reduction='none') test


In [62]:
class A():
    def reset(self):
        print("A reseted")


class B():
    def reset(self):
        print("B reseted")

class State:
    def __init__(self, metric_meters):
        self.metric_meters = metric_meters

metric_meters = [A(), A(), B(), B(), B(), A()]
print(metric_meters)
state = State(metric_meters)

state.metric_meters = [m for m in state.metric_meters if isinstance(m, A)]
for metric in state.metric_meters:
    metric.reset()

# omelist = [x for x in somelist if not determine(x)]

# for i, metric in enumerate(state.metric_meters):
#     if isinstance(metric, A):
#         metric.reset()
#     elif isinstance(metric, B):
#         print("Deleting B")
#         del state.metric_meters[i]
print(state.metric_meters)
        
    

[<__main__.A object at 0x7f59d08592e8>, <__main__.A object at 0x7f59d0859860>, <__main__.B object at 0x7f59d08597f0>, <__main__.B object at 0x7f59d08598d0>, <__main__.B object at 0x7f59d0859898>, <__main__.A object at 0x7f59d0859828>]
A reseted
A reseted
A reseted
[<__main__.A object at 0x7f59d08592e8>, <__main__.A object at 0x7f59d0859860>, <__main__.A object at 0x7f59d0859828>]


In [51]:
import functools
METRIC_FROM_NAME = {
    # Full Reference
    "psnr": functools.partial(
        piq.psnr),
}

def get_metric(name, reduction='none'):
    metric = METRIC_FROM_NAME[name]
    return functools.partial(metric, reduction=reduction)
    
prediction = torch.rand(4, 3, 128, 128)
target = torch.rand(4, 3, 128, 128)

get_metric("psnr")(prediction, target)

tensor(7.7687)

In [55]:
a = [piq.ContentLoss(), piq.StyleLoss(), piq.LPIPS(), piq.DISTS(), piq.GMSDLoss()]

In [ ]:
[07-24 20:48] - Best loss: 0.0394
[07-24 20:48] - Best psnr: 25.0806
[07-24 20:48] - Best ssim: 0.7546
[07-24 20:48] - Best ms-ssim: 0.9332
[07-24 20:48] - Best gmsd: 0.0846
[07-24 20:48] - Best ms-gmsd: 0.0816
[07-24 20:48] - Best ms-gmsdc: 0.0681
[07-24 20:48] - Best fsim: 0.8802
[07-24 20:48] - Best fsimc: 0.8744
[07-24 20:48] - Best vsi: 0.9544
[07-24 20:48] - Best mdsi: 0.4146
[07-24 20:48] - Best vifp: 0.9085
[07-24 20:48] - Best content_vgg16: 0.2474
[07-24 20:48] - Best content_vgg19: 0.2679
[07-24 20:48] - Best content_vgg16_ap: 0.2176
[07-24 20:48] - Best content_vgg19_ap: 0.2257
[07-24 20:48] - Best style_vgg16: 9222874.0769
[07-24 20:48] - Best style_vgg19: 7961090.4615
[07-24 20:48] - Best lpips: 0.2275
[07-24 20:48] - Best dists: 0.1868
[07-24 20:48] - Best brisque: 37.7652

In [ ]:
[07-27 15:22] - Best loss: 0.0353
[07-27 15:22] - Best psnr: 25.4831
[07-27 15:22] - Best ssim: 0.7822
[07-27 15:22] - Best ms-ssim: 0.9491
[07-27 15:22] - Best gmsd: 0.0853
[07-27 15:22] - Best ms-gmsd: 0.0827
[07-27 15:22] - Best ms-gmsdc: 0.0693
[07-27 15:22] - Best fsim: 0.8753
[07-27 15:22] - Best fsimc: 0.8733
[07-27 15:22] - Best vsi: 0.9596
[07-27 15:22] - Best mdsi: 0.4467
[07-27 15:22] - Best vifp: 0.8642
[07-27 15:22] - Best content_vgg16_ap: 0.1814
[07-27 15:22] - Best style_vgg16: 147372154.8571
[07-27 15:22] - Best lpips: 0.2060
[07-27 15:22] - Best dists: 0.1557
[07-27 15:22] - Best brisque: 45.1469
[07-27 15:22] - Best is_metric_vgg16: 1.1791
[07-27 15:22] - Best is_vgg16: 0.4978
[07-27 15:22] - Best kid_vgg16: 0.0180
[07-27 15:22] - Best fid_vgg16: 14.8280
[07-27 15:22] - Best msid_vgg16: 8.8804

In [ ]:
[07-27 16:27] - Best loss: 0.0036
[07-27 16:27] - Best psnr: 25.3338
[07-27 16:27] - Best ssim: 0.7586
[07-27 16:27] - Best ms-ssim: 0.9371
[07-27 16:27] - Best gmsd: 0.0873
[07-27 16:27] - Best ms-gmsd: 0.0832
[07-27 16:27] - Best ms-gmsdc: 0.0696
[07-27 16:27] - Best fsim: 0.8803
[07-27 16:27] - Best fsimc: 0.8775
[07-27 16:27] - Best vsi: 0.9612
[07-27 16:27] - Best mdsi: 0.4124
[07-27 16:27] - Best vifp: 0.9856
[07-27 16:27] - Best content_vgg16_ap: 0.1994
[07-27 16:27] - Best style_vgg16: 56819345.4286
[07-27 16:27] - Best lpips: 0.2112
[07-27 16:27] - Best dists: 0.1431
[07-27 16:27] - Best brisque: 36.5921
[07-27 16:27] - Best is_metric_vgg16: 1.2533
[07-27 16:27] - Best is_vgg16: 0.3236
[07-27 16:27] - Best kid_vgg16: 0.0104
[07-27 16:27] - Best fid_vgg16: 11.7871
[07-27 16:27] - Best msid_vgg16: 4.0915
[07-27 16:27] - Finished training!

In [ ]:
[07-27 17:22] - Best loss: 0.0567
[07-27 17:22] - Best psnr: 24.6242
[07-27 17:22] - Best ssim: 0.7714
[07-27 17:22] - Best ms-ssim: 0.9410
[07-27 17:22] - Best gmsd: 0.0834
[07-27 17:22] - Best ms-gmsd: 0.0799
[07-27 17:22] - Best ms-gmsdc: 0.0663
[07-27 17:22] - Best fsim: 0.8866
[07-27 17:22] - Best fsimc: 0.8804
[07-27 17:22] - Best vsi: 0.9596
[07-27 17:22] - Best mdsi: 0.4374
[07-27 17:22] - Best vifp: 0.9039
[07-27 17:22] - Best content_vgg16_ap: 0.2017
[07-27 17:22] - Best style_vgg16: 290767860.0000
[07-27 17:22] - Best lpips: 0.2258
[07-27 17:22] - Best dists: 0.1545
[07-27 17:22] - Best brisque: 34.5464
[07-27 17:22] - Best is_metric_vgg16: 1.3126
[07-27 17:22] - Best is_vgg16: 0.3683
[07-27 17:22] - Best kid_vgg16: 0.0117
[07-27 17:22] - Best fid_vgg16: 13.4920
[07-27 17:22] - Best msid_vgg16: 7.6096
[07-27 17:22] - Finished training!